In [2]:
from extract_parquet import read_parquet,extract_file_path
import cmbagent 
from make_prompts import make_prompt, extract_python_definition
import os
import pandas as pd

In [3]:
%load_ext autoreload
%autoreload 2
from ins import evaluate_camb

In [5]:
# Example with a single file
df = pd.read_csv('data/camb_data_v2.csv')


In [8]:
df

,Unnamed: 0,statement,prompt,reference_code,output
0,0,Compute the CMB Temperature power spectrum for...,###Problem\nCompute the CMB temperature power ...,import camb\nimport numpy as np\ndef get_tt(om...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
1,1,Compute the CMB Temperature power spectrum for...,### Problem:\nCompute the CMB temperature powe...,import camb\nimport numpy as np\ndef get_tt(om...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
2,2,Compute the CMB Temperature power spectrum for...,### Problem:\nCompute the CMB temperature powe...,import camb\nimport numpy as np\ndef get_tt(H0...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
3,3,Compute the Raw CMB Temperature power spectrum...,### Problem:\nCompute the raw CMB temperature ...,import camb\nimport numpy as np\ndef get_tt(H0...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
4,4,Compute the CMB E-mode power spectrum for Plan...,### Problem:\nCompute the CMB E-mode power spe...,import camb\nimport numpy as np\ndef get_ee(ta...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
5,5,Compute the total CMB B-mode power spectrum fo...,### Problem:\nCompute the CMB B-mode power spe...,import camb\nimport numpy as np\ndef get_bb(r=...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
6,6,Compute the total CMB B-mode power spectrum fo...,### Problem:\nCompute the CMB B-mode power spe...,import camb\nimport numpy as np\ndef get_bb(r=...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."
7,7,Compute the angular diameter distance for Plan...,### Problem:\nCompute the angular diameter dis...,import camb\nimport numpy as np\ndef get_angul...,"(array([0. , 0.04040404, 0.08080808, 0...."
8,8,Compute the linear matter power spectrum with ...,### Problem:\nCompute the linear matter power ...,import camb\nimport numpy as np\ndef get_matte...,"(array([1.00000000e-04, 1.04737090e-04, 1.0969..."
9,9,Compute the delensed total CMB B-mode power sp...,### Problem:\nCompute the Delensed CMB B-mode ...,import camb\nimport numpy as np\ndef get_bb_de...,"(array([ 0, 1, 2, ..., 2998, 2999, 300..."


In [10]:
# --- usage ---------------------------------------------------------------
i = 9                                        # whichever row you want
prompt = df.prompt.iloc[i]

In [12]:
reference_code = df.reference_code.iloc[i]

In [14]:
reference_code

"import camb\nimport numpy as np\ndef get_bb_delensed(r=0.1,delens_eff=10):\n    omk=0\n    omch2=0.122\n    mnu=0.06\n    ombh2=0.022\n    H0 = 67.5\n    tau = 0.06\n    lmax=3000\n    pars = camb.set_params(H0=H0, ombh2=ombh2, omch2=omch2, mnu=mnu, omk=omk, tau=tau, r=r,lmax=lmax+500)\n    pars.WantTensors = True\n    results = camb.get_results(pars)\n    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')\n    spectra_tensor = powers['tensor']\n    Alens = 1 - (delens_eff/100)\n    spectra =results.get_partially_lensed_cls(Alens,CMB_unit='muK',lmax=lmax+500)\n    return np.arange(lmax+1), spectra[:lmax+1,2] + spectra_tensor[:lmax+1,2]\n"

In [16]:
results = cmbagent.one_shot(prompt,
                   max_rounds=50,
                   #agent='camb_context',
                   agent='engineer',
                   #engineer_model='gpt-4.1',
                   #researcher_model='gpt-4.1',
                   #engineer_model='gemini-2.5-pro-exp-03-25',
                            
                   work_dir="./evals",
                  )

_User (to chat_manager):

### Problem:
Compute the Delensed CMB B-mode power spectra in uK^2 units for tensor to scalar ratio set to 0.1, up to a maximum multipole of 3000. Consider other parameters such as omk=0, omch2=0.122, mnu=0.06, ombh2=0.022, H0=67.5 tau=0.06. Use a delensing efficiency of 10%.

### Requirements
provide a python code with r and delens_eff as variables
\begin{python}
def get_bb_delensed(r=0.1,delens_eff=10):
    return l, cl
\end{python}


--------------------------------------------------------------------------------

Next speaker: engineer


>>>>>>>> USING AUTO REPLY...
             Model    agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-2025-04-14 engineer 0.01227           1871               1066          2937
engineer (to chat_manager):

**Code Explanation:**

This code defines a function `get_bb_delensed` that computes the delensed CMB B-mode (BB) power spectrum up to multipole l=3000, for a given tensor-to-scalar ratio `r` and delen

In [18]:
filepath = os.path.join(results['final_context']['work_dir'],extract_file_path(results['final_context']['current_codebase']))

In [19]:
filepath

'./evals/codebase/get_bb_delensed.py'

In [20]:
with open(filepath, "r", encoding="utf-8") as f:
    candidate_code = f.read()

In [25]:
evaluate_camb(reference_code,candidate_code)

╭─ camb_with_similarity (1 sample): openai/gpt-4.1-2025-04-14 ────────────────────────────────────────────────────╮
│ reference: import camb                                                                       dataset: (samples) │
│ import numpy as np                                                                                              │
│ def get_bb_delensed(r=0.1,delens_eff=10):                                                                       │
│     omk=0                                                                                                       │
│     omch2=0.122                                                                                                 │
│     mnu=0.06                                                                                                    │
│     ombh2=0.022                                                                                                 │
│     H0 = 67.5                                                                                                   │
│     tau = 0.06                                                                                                  │
│     lmax=3000                                                                                                   │
│     pars = camb.set_params(H0=H0, ombh2=ombh2, omch2=omch2, mnu=mnu, omk=omk, tau=tau,                          │
│ r=r,lmax=lmax+500)                                                                                              │
│     pars.WantTensors = True                                                                                     │
│     results = camb.get_results(pars)                                                                            │
│     powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')                                                │
│     spectra_tensor = powers['tensor']                                                                           │
│     Alens = 1 - (delens_eff/100)                                                                                │
│     spectra =results.get_partially_lensed_cls(Alens,CMB_unit='muK',lmax=lmax+500)                               │
│     return np.arange(lmax+1), spectra[:lmax+1,2] + spectra_tensor[:lmax+1,2]                                    │
│ , candidate: # filename: codebase/get_bb_delensed.py                                                            │
│ import numpy as np                                                                                              │
│ import camb                                                                                                     │
│ from camb import model, initialpower                                                                            │
│                                                                                                                 │
│ def get_bb_delensed(r=0.1, delens_eff=10):                                                                      │
│     r"""                                                                                                        │
│     Compute the delensed CMB B-mode (BB) power spectrum up to l=3000 for a given                                │
│ tensor-to-scalar ratio and delensing efficiency.                                                                │
│                                                                                                                 │
│     Parameters                                                                                                  │
│     ----------                                                                                                  │
│     r : float, optional                                                                                         │
│         Tensor-to-scalar ratio (dimensionless, default 0.1).                                                    │
│     delens_eff : float, optional                      

{'no_input_array_io': 1.0, 'content_similarity': 0.37907181486605457}